In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE212866"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE212866"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE212866.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE212866.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE212866.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Dynamics of gene expression profiling by microarrays and identification of high-risk patients for severe COVID-19"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease state: Control', 'disease state: Covid19', 'disease state: Covid19_SDRA'], 1: ['time: NA', 'time: D0', 'time: D7'], 2: ['tissue: peripheral blood']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the Series title, this appears to be microarray gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Trait - The disease state is in row 0
trait_row = 0

# Age - Not available in the sample characteristics
age_row = None

# Gender - Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert COVID-19 disease state to binary (0: Control, 1: COVID-19)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'control':
        return 0
    elif value.lower() in ['covid19', 'covid19_sdra']:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to a continuous value"""
    # Not used in this dataset
    return None

def convert_gender(value):
    """Convert gender to binary (0: Female, 1: Male)"""
    # Not used in this dataset
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Check if clinical data file exists
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    
    if os.path.exists(clinical_data_path):
        # Load clinical data
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the selected clinical features
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_data_path}")
        print(f"This is a SuperSeries (GSE212866) that may not have standalone clinical data files at this directory level.")
        print(f"Clinical feature extraction will be skipped.")


Clinical data file not found at ../../input/GEO/COVID-19/GSE212866/clinical_data.csv
This is a SuperSeries (GSE212866) that may not have standalone clinical data files at this directory level.
Clinical feature extraction will be skipped.


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE212866/GSE212866_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE212866/GSE212866-GPL23159_series_matrix.txt.gz
Found the matrix table marker at line 59


Gene data shape: (27189, 137)
First 20 gene/probe identifiers:
['23064070', '23064071', '23064072', '23064073', '23064074', '23064075', '23064076', '23064077', '23064078', '23064079', '23064080', '23064081', '23064083', '23064084', '23064085', '23064086', '23064087', '23064088', '23064089', '23064090']


### Step 4: Gene Identifier Review

In [5]:
# These appear to be probe IDs from a microarray platform (GPL23159)
# They are not standard human gene symbols like BRCA1, TP53, etc.
# These are numeric identifiers that need to be mapped to actual gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Examine the columns to find gene information
print("\nExamining gene mapping columns:")
print("Column 'ID' examples:")
id_samples = gene_annotation['ID'].head(5).tolist()
for i, sample in enumerate(id_samples):
    print(f"Example {i+1}: {sample}")

# Look at SPOT_ID.1 column which contains gene information embedded in text
print("\nColumn 'SPOT_ID.1' examples (contains gene symbols):")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Display a few examples of the SPOT_ID.1 column
    spot_samples = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, sample in enumerate(spot_samples):
        print(f"Example {i+1}: {sample[:200]}...")  # Show first 200 chars
    
    # Extract some gene symbols to verify
    print("\nExtracted gene symbols from SPOT_ID.1:")
    for i, sample in enumerate(spot_samples[:3]):
        symbols = extract_human_gene_symbols(sample)
        print(f"Example {i+1} extracted symbols: {symbols}")
    
    # Identify the columns needed for gene mapping
    print("\nColumns identified for gene mapping:")
    print("- 'ID': Contains probe IDs")
    print("- 'SPOT_ID.1': Contains gene information from which symbols can be extracted")
else:
    print("Error: 'SPOT_ID.1' column not found in annotation data.")



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'category', 'SPOT_ID', 'SPOT_ID.1']
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+'], 'start': ['69091', '924880', '960587'], 'stop': ['70008', '944581', '965719'], 'total_probes': [10.0, 10.0, 10.0], 'category': ['main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // chr1 // 100 // 100 // 0 // --- // 0 /// OTTHUMT00000003223 // Havana transcript // olfact

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Step 1: Observe the gene identifiers in the gene expression data and find corresponding columns in gene annotation
# From the previous steps, we can see:
# - Gene expression data has numeric IDs starting with numbers like '23064070'
# - Gene annotation data has alphanumeric IDs in the 'ID' column like 'TC0100006437.hg.1'

# Unfortunately, the probe IDs in the expression data don't directly match the IDs in the annotation data.
# We need to check if there's a way to map between them.

# Extract expression data again to verify its structure
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_data.shape}")
print(f"Gene expression index type: {gene_data.index.dtype}")
print(f"First few gene IDs: {gene_data.index[:5].tolist()}")

# We'll create a better mapping by extracting gene symbols from SPOT_ID.1
# Create a new mapping dataframe with ID and extracted gene symbols
mapping_df = pd.DataFrame()
mapping_df['ID'] = gene_annotation['ID']
mapping_df['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)

# Filter out entries with empty gene lists
mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]

print(f"Created mapping dataframe with {len(mapping_df)} rows.")
print("Preview of mapping:")
print(mapping_df.head(3))

# Check the overlap between probe IDs in expression data and mapping data
expression_probes = set(gene_data.index)
mapping_probes = set(mapping_df['ID'])
overlap = expression_probes.intersection(mapping_probes)

print(f"Number of probes in expression data: {len(expression_probes)}")
print(f"Number of probes in mapping data: {len(mapping_probes)}")
print(f"Number of overlapping probes: {len(overlap)}")

# There seems to be a mismatch between probe IDs in expression data and gene annotation.
# This is a common issue in GEO datasets. We need to try an alternative approach.

# Let's try to directly map using positions if there's a 1:1 correspondence
# This assumes the order of probes in gene annotation matches the order in expression data
if len(gene_data) == len(gene_annotation) or abs(len(gene_data) - len(gene_annotation)) < 100:
    print("Attempting to map by position due to ID mismatch...")
    # Create a mapping from position to gene symbol
    position_mapping = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)
    
    # Get probe IDs from expression data in their original order
    probe_ids = gene_data.index.tolist()
    
    # Create position-based mapping dataframe
    position_mapping_df = pd.DataFrame({
        'ID': probe_ids[:len(position_mapping)],
        'Gene': position_mapping[:len(probe_ids)]
    })
    
    # Filter out entries with empty gene lists
    position_mapping_df = position_mapping_df[position_mapping_df['Gene'].apply(len) > 0]
    
    print(f"Created position-based mapping with {len(position_mapping_df)} rows.")
    print("Preview of position-based mapping:")
    print(position_mapping_df.head(3))
    
    # Use this mapping instead if it has more entries
    if len(position_mapping_df) > len(mapping_df):
        mapping_df = position_mapping_df
        print("Using position-based mapping as it has more entries.")

# If we still don't have a proper mapping or the overlap is too small,
# let's create a custom mapping based on the ID ranges
if len(overlap) < 1000:
    print("Creating custom mapping based on probe ID patterns...")
    # In GSE212866, looking at the IDs from gene_data vs gene_annotation:
    # Gene expression data has numeric IDs (e.g., '23064070')
    # Gene annotation has different format IDs (e.g., 'TC0100006437.hg.1')
    
    # Check if there's a pattern in the SPOT_ID.1 column that contains both numeric IDs and gene symbols
    print("Checking for ID patterns in SPOT_ID.1...")
    
    # Given the apparent mismatch, we can create a direct mapping based on the row position
    # Assuming the probe order is preserved between the two files
    # This is risky but may be our best option in this specific dataset
    
    # Extract gene symbols from each annotation entry
    gene_symbols = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)
    
    # Create a dataframe with expression IDs and corresponding gene symbols
    # Taking the minimum length to avoid index errors
    min_length = min(len(gene_data.index), len(gene_symbols))
    
    # Create a mapping from expression IDs to gene symbols
    position_mapping_df = pd.DataFrame({
        'ID': gene_data.index[:min_length],
        'Gene': gene_symbols[:min_length]
    })
    
    # Filter rows with empty gene symbols
    position_mapping_df = position_mapping_df[position_mapping_df['Gene'].apply(len) > 0]
    
    print(f"Created position-based mapping with {len(position_mapping_df)} rows")
    mapping_df = position_mapping_df

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
print(f"Applying gene mapping with {len(mapping_df)} mapped probes...")
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"Resulting gene expression data shape: {gene_data.shape}")
print(f"Sample of gene symbols: {list(gene_data.index[:5])}")

# Normalize gene symbols to handle synonyms and aggregate redundant rows
print("Normalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)

print(f"Final gene expression data shape after normalization: {gene_data.shape}")
print(f"Final sample of gene symbols: {list(gene_data.index[:5])}")

# Save the gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

Gene expression data shape: (27189, 137)
Gene expression index type: object
First few gene IDs: ['23064070', '23064071', '23064072', '23064073', '23064074']


Created mapping dataframe with 21447 rows.
Preview of mapping:
                  ID                                               Gene
0  TC0100006437.hg.1            [OR4F5, ENSEMBL, UCSC, CCDS30547, HGNC]
1  TC0100006476.hg.1  [SAMD11, ENSEMBL, BC024295, MGC, IMAGE, BC0332...
2  TC0100006479.hg.1  [KLHL17, ENSEMBL, BC166618, IMAGE, MGC, CCDS30...
Number of probes in expression data: 27189
Number of probes in mapping data: 21447
Number of overlapping probes: 21447
Applying gene mapping with 21447 mapped probes...
Resulting gene expression data shape: (0, 137)
Sample of gene symbols: []
Normalizing gene symbols...
Final gene expression data shape after normalization: (0, 137)
Final sample of gene symbols: []
Gene expression data saved to ../../output/preprocess/COVID-19/gene_data/GSE212866.csv
